<a href="https://colab.research.google.com/github/seunghyeon2776/98/blob/master/%5B2021_02_19%5D_new_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import os
import glob
import cv2



train_img_path = "/content/drive/Shareddrives/CTRC-OralDetect-Project/HM_Color/train"
test_img_path = "/content/drive/Shareddrives/CTRC-OralDetect-Project/HM_Color/test"

classes = ['1.Cancer', '2.Precancer', '3.Inflammatory', '4.Normal']
image_size=224

In [ ]:
#train_img

images1 = []
labels1 = []
ids1 = []
cls1 = []

print('Reading cancer training images')
for fld in classes:
    index = classes.index(fld)
    print('Loading {} files (Index: {})'.format(fld, index))
    path = os.path.join(train_img_path, fld, '*g')
    files = glob.glob(path)
    for fl in files:
        image = cv2.imread(fl)
        image = cv2.resize(image, (image_size, image_size), cv2.INTER_LINEAR)
        images1.append(image)
        label = np.zeros(len(classes))
        label[index] = 1.0
        labels1.append(label)
        flbase = os.path.basename(fl)
        ids1.append(flbase)
        cls1.append(fld)
images1 = np.array(images1)
labels1 = np.array(labels1)
ids1 = np.array(ids1)
cls1 = np.array(cls1)

Reading cancer training images
Loading 1.Cancer files (Index: 0)
Loading 2.Precancer files (Index: 1)


In [ ]:
#test_img

images2 = []
labels2 = []
ids2 = []
cls2 = []

print('Reading cancer testing images')
for fld in classes:
    index = classes.index(fld)
    print('Loading {} files (Index: {})'.format(fld, index))
    path = os.path.join(test_img_path, fld, '*g')
    files = glob.glob(path)
    for fl in files:
        image = cv2.imread(fl)
        image = cv2.resize(image, (image_size, image_size), cv2.INTER_LINEAR)
        images2.append(image)
        label = np.zeros(len(classes))
        label[index] = 1.0
        labels2.append(label)
        flbase = os.path.basename(fl)
        ids2.append(flbase)
        cls2.append(fld)
images2 = np.array(images2)
labels2 = np.array(labels2)
ids2 = np.array(ids2)
cls2 = np.array(cls2)

In [4]:
from sklearn.utils import shuffle

images1, labels1 = shuffle(images1, labels1)
images2, labels2 = shuffle(images2, labels2)

X_train = images1
Y_train = labels1
X_test = images2
Y_test = labels2

In [5]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

In [6]:
'''
X_train = X_train.reshape(X_train.shape[0], 4, 224, -1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 4, 224, -1).astype('float32')
'''

# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

# one hot encode outputs
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)
num_classes = Y_test.shape[1]

'''
Y_train = Y_train.reshape(Y_train.shape[0], 4, 2, -1).astype('float32')
Y_test = Y_test.reshape(Y_test.shape[0], 4, 2, -1).astype('float32')

Y_train = Y_train / 255
X_train = X_train / 255
'''

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

print(X_train.shape)
print(Y_train.shape)


(2361, 224, 224, 3)
(2361, 4, 2)


In [7]:
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras import optimizers, initializers, regularizers, metrics
import tensorflow as tf

def cnn_model():

    model = models.Sequential()
    model.add(layers.Dense(2048, kernel_regularizer = regularizers.l2
                                  (0.001),activation='relu'))
    model.add(layers.Dense(2048, kernel_regularizer = regularizers.l2
                                  (0.001),activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add( MaxPooling2D((2,2), padding='same'))


    #Layer weight regularizers    #kernel_regularizer: Regularizer to apply a penalty on the layer's kerne 
    #L1: float; L1 regularization factor = 0.001   
    #L1 regularization : 가중치의 절댓값에 비례하는 비용이 추가됨(가중치의 L1 norm)
    model.add(layers.Dense(1024, kernel_regularizer = regularizers.l2
                                  (0.001),activation='relu'))
    model.add(layers.Dense(1024, kernel_regularizer = regularizers.l2
                                  (0.001),activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add( MaxPooling2D((2,2), padding='same'))

  
    #L2: float; L2 regularization factor = 0.001  
    #L2 regularization(=weight decay) : 가중치의 제곱에 비례하는 비용이 추가됨(가중치의 L2 norm)
    model.add(layers.Dense(512, kernel_regularizer = regularizers.l2
                                  (0.001),activation='relu'))
    model.add(layers.Dense(512, kernel_regularizer = regularizers.l2
                                  (0.001),activation='relu'))
    model.add(layers.Dense(512, kernel_regularizer = regularizers.l2
                                  (0.001),activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add( MaxPooling2D((2,2), padding='same'))


    model.add(layers.Dense(256, kernel_regularizer = regularizers.l2
                                  (0.001),activation='relu'))
    model.add(layers.Dense(256, kernel_regularizer = regularizers.l2
                                  (0.001),activation='relu'))
    model.add(layers.Dense(256, kernel_regularizer = regularizers.l2
                                  (0.001),activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add( MaxPooling2D((2,2), padding='same'))


    model.add(layers.Dense(128, kernel_regularizer = regularizers.l2
                                  (0.001),activation='relu'))
    model.add(layers.Dense(128, kernel_regularizer = regularizers.l2
                                  (0.001),activation='relu'))
    model.add(layers.Dense(128, kernel_regularizer = regularizers.l2
                                  (0.001),activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add( MaxPooling2D((2,2), padding='same'))

    model.add(Flatten()) 
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(4,activation='relu'))


    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    
    return model



In [8]:
# build the model
model = cnn_model()
# Fit the model
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=32, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))


Epoch 1/10


ResourceExhaustedError: ignored

In [ ]:
# train_set 은 튜플로 두 개의 ndsarray를 원소로 가지고 있음
print train_set
print type(train_set)
print len(train_set)

# 튜플을 이렇게 둘로 나눌 수 있음
X_train, y_train = train_set
X_test, y_test = test_set

print type(X_train)
print len(X_train)
print len(y_train)
print X_train
print y_train


In [6]:
from keras.applications.resnet50 import ResNet50
import tensorflow

model=ResNet50(weights='imagenet',
               input_shape=(320,320,3),
               include_top=False)

KeyboardInterrupt: ignored